In [1]:
# Needed for functions
import numpy as np
import time
from copy import deepcopy

# Import Qiskit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise

# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
import qiskit.ignis.mitigation.measurement as mc

In [2]:
q = QuantumRegister(2)
c = ClassicalRegister(2)
circ = QuantumCircuit(q, c)

circ.h(0)
circ.cx(q[0], q[1])

job = qiskit.execute(circ, Aer.get_backend('statevector_simulator'))
psi = job.result().get_statevector(circ)
print(psi)

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [3]:
q2 = QuantumRegister(6)
circ1 = QuantumCircuit(q2)
circ1.h(q2[3])
circ1.cx(q2[3], q2[5])
print(circ1)

                   
q1_0: |0>──────────
                   
q1_1: |0>──────────
                   
q1_2: |0>──────────
         ┌───┐     
q1_3: |0>┤ H ├──■──
         └───┘  │  
q1_4: |0>───────┼──
              ┌─┴─┐
q1_5: |0>─────┤ X ├
              └───┘


In [4]:
# Generate circuits and run on simulator
t = time.time()
# Generate the state tomography circuits. Only pass in the 
# registers we want to measure (in this case 3 and 5)
qst_circ = state_tomography_circuits(circ1, [q2[3],q2[5]])
job = qiskit.execute(qst_circ, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_bell = StateTomographyFitter(job.result(), qst_circ)
print('Time taken:', time.time() - t)

Time taken: 0.3058938980102539


In [5]:
rho_bell = tomo_bell.fit()
print(rho_bell)

[[ 0.49955096+0.j          0.00151405-0.0001381j  -0.00088404-0.00642248j
   0.49588745-0.00799029j]
 [ 0.00151405+0.0001381j   0.00168417+0.j         -0.0006168 -0.001584j
   0.00308251-0.00218284j]
 [-0.00088404+0.00642248j -0.0006168 +0.001584j    0.00176693+0.j
   0.00078493+0.00870048j]
 [ 0.49588745+0.00799029j  0.00308251+0.00218284j  0.00078493-0.00870048j
   0.49699794+0.j        ]]
